In [45]:
import pandas as pd

In [46]:
df = pd.read_csv('data/LabelledData.csv',sep='\t')
df.head()

,id,question,category
0,1,how did serfdom develop in and then leave russ...,unknown
1,2,what films featured the character popeye doyle ?,what
2,3,how can i find a list of celebrities ' real na...,unknown
3,4,what fowl grabs the spotlight after the chines...,what
4,5,what is the full form of .com ?,what


In [47]:
print("size of dataset:{}".format(len(df)))
df['category'].value_counts()

size of dataset:1483


what           609
who            402
unknown        272
affirmation    104
when            96
Name: category, dtype: int64

In [48]:
"overall information of data"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1483 entries, 0 to 1482
Data columns (total 3 columns):
id          1483 non-null int64
question    1483 non-null object
category    1483 non-null object
dtypes: int64(1), object(2)
memory usage: 34.9+ KB


In [49]:
"Mapping the categories to a numerical value"
df['category']=df['category'].map({'what':0,'who':1,'affirmation':2,'when':3,'unknown':4})

In [50]:
df.head()

,id,question,category
0,1,how did serfdom develop in and then leave russ...,4
1,2,what films featured the character popeye doyle ?,0
2,3,how can i find a list of celebrities ' real na...,4
3,4,what fowl grabs the spotlight after the chines...,0
4,5,what is the full form of .com ?,0


In [51]:
X =  df['question'].values.tolist()
Y = df['category'].values.tolist()

In [52]:
df_test = pd.read_csv('data/TestData.csv',sep='\t',header=None)

df_test.head()

,0
0,how did serfdom develop in and then leave russ...
1,what films featured the character popeye doyle ?
2,how can i find a list of celebrities ' real na...
3,what fowl grabs the spotlight after the chines...
4,what is the full form of .com ?


In [53]:
x_test = df_test[0].values.tolist()
corpus = X+x_test

In [54]:
token_corpus = [i.split(' ') for i in corpus]

In [55]:
"""Training a word2vec model to generate word embeddings"""
import gensim
from gensim.models import Word2Vec
model = Word2Vec(token_corpus,min_count=1,window=5)


In [56]:
X = [[model.wv[word] for word in sent.split(' ')]for sent in X]   

In [57]:
"""Splitting the dataset to train and validation data """
import keras
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(X,Y,test_size=0.2, random_state=42)
x_train = pad_sequences(x_train, maxlen=4, padding='post', truncating='post', value=0.0)
x_val = pad_sequences(x_val, maxlen=4, padding='post', truncating='post', value=0.0)

In [58]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
x_val = x_val.reshape(x_val.shape[0],x_val.shape[1]*x_val.shape[2])

In [59]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [60]:
y_pred = clf.predict(x_val)

# Acccuracy on validation data

In [61]:
from sklearn import metrics
metrics.accuracy_score(y_val,y_pred)

0.9259259259259259

In [62]:
metrics.confusion_matrix(y_val,y_pred)

array([[110,   1,   0,   5,   0],
       [  0,  86,   1,   0,   0],
       [  0,   3,  13,   0,   3],
       [  3,   0,   0,  19,   0],
       [  2,   1,   3,   0,  47]])

In [68]:
x_test = [[model.wv[word] for word in sent.split(' ')]for sent in x_test]   
x_test = pad_sequences(x_test, maxlen=4, padding='post', truncating='post', value=0.0)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])

In [69]:
y_test_pred=clf.predict(x_test)

In [71]:
res_df = pd.DataFrame()
res_df['question'] = df_test[0]
res_df['category'] = y_test_pred

In [75]:
res_df['category']=res_df['category'].map({0:'what',1:'who',2:'affirmation',3:'when',4:'unknown'})
res_df.head()

,question,category
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [76]:
res_df.to_csv('results.csv',header=None)